<a href="https://colab.research.google.com/github/emanuelemorales/Market-Basket-Analysis/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fedema","key":"421f607430abf02a32557ca691e32d9a"}'}

In [ ]:
#import libraries
import itertools

# Spark Setting

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 44.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=34dc59a819a20ffdfb79f57f29532cea664a9108ed2c6e97d027a3e5e35906ae
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


Initialize a SparContext and a SparkSession

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_set, col, count
from pyspark.sql import SQLContext

In [ ]:
conf = SparkConf().setAppName("MarketBasket")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

##Mount Drive and import Dataset

Install and Download Kaggle API's Key

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
try :  
    import kaggle  
      
except OSError as error :  
    print(error)
    ! pip install -q kaggle

Download the Dataset

In [ ]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset

100% 1.44G/1.44G [00:17<00:00, 40.9MB/s]
100% 1.44G/1.44G [00:17<00:00, 86.0MB/s]


Import the Dataset in the COLAB work environment

In [ ]:
from zipfile import ZipFile
file_name = "imdb-dataset.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


##Filter actor and movies by using sqlContext

From title principals table, filter just actor and actress, that will be the items of the analysis.
Then select just the columns of interest: tconst (the movies' code) and nconst (the actor's code).

In [ ]:
sqlContext = SQLContext(sc)

title_principals = 'title.principals.tsv.gz'
title_principals = sqlContext.read.csv(title_principals, header=True, sep = '\t')

title_principals.show(5)

title_principals = title_principals.filter((title_principals.category == "actor") | (title_principals.category == "actress"))
title_principals = title_principals.select(col("tconst"),col("nconst"))

+---------+--------+---------+---------------+--------------------+-----------+
|   tconst|ordering|   nconst|       category|                 job| characters|
+---------+--------+---------+---------------+--------------------+-----------+
|tt0000001|       1|nm1588970|           self|                  \N|["Herself"]|
|tt0000001|       2|nm0005690|       director|                  \N|         \N|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|         \N|
|tt0000002|       1|nm0721526|       director|                  \N|         \N|
|tt0000002|       2|nm1335271|       composer|                  \N|         \N|
+---------+--------+---------+---------------+--------------------+-----------+
only showing top 5 rows



From title basics table filter just the movies, that will be the baskets of the analysis.

In [ ]:
title_basics = 'title.basics.tsv.gz'
title_basics = sqlContext.read.csv(title_basics, header=True, sep = '\t')

title_basics.show(5) # print example

#Filter by movies
title_basics = title_basics.filter((title_basics.titleType == "movie"))

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

With a left join operation, combine title principle and title basic table in order to obtain just the actors appearing in movies:

In [ ]:
title_principals = title_principals.join(title_basics, ["tconst"], 'leftsemi')
title_principals.show(5) # print example

+---------+---------+
|   tconst|   nconst|
+---------+---------+
|tt0002591|nm0029806|
|tt0002591|nm0509573|
|tt0003689|nm0694718|
|tt0003689|nm0101071|
|tt0003689|nm0910564|
+---------+---------+
only showing top 5 rows



In [ ]:
#Import name_basics table
name_basics_path = 'name.basics.tsv.gz'
name_basics = sqlContext.read.csv(name_basics_path, header = True, sep = '\t')
name_basics.show(5)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00599...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,soun...|tt0078723,tt00804...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



#Implementation of a-priori and SON algorithms

In [ ]:
#DEFINITION OF FUNCTIONS:

#Definition of sum function
def sum_actors(x,y):
    return x+y

#Definition of filtering functions: it checks if a set of item is a subset of a basket.
def filtering(rddlist, filt):
  for item in filt:
    if set(list(item)).issubset(set(rddlist)):
      return ((item, 1))

#A-Priori algorithm: 

Implementation of A-Priori algorithm to retrieve the frequent pairs of actors.
The function takes in input an RDD and a threshold.
The function is composed by the combination of the main Spark's functions: map, reduce and filter.

In [ ]:
def apriori(rdd, threshold):
  
  flat_list  = rdd.flatMap(list) 

  singleton = flat_list.map(lambda item: (item , 1)) #add one for each actor appearence
  singleton_summed = singleton.reduceByKey(sum_actors) #sum of values by actor as key
  singleton_filtered = singleton_summed.filter(lambda item: item[1] >= threshold ) #consider items that appear singularly a number of time larger than the threshold in the baskets.

  #obtain a list of the codes of the items
  frequent_actors = singleton_filtered.map(lambda item: (item[0]))

  #Obtain all the pairs of frequent items:
  pairs_list = list(itertools.combinations(frequent_actors.toLocalIterator(),2))

  #Create the support table for the pairs of items by applying the filtering function previously created.
  support_table_pairs = rdd.map(lambda x : filtering(x, pairs_list)).filter(lambda x: x is not None).cache() #Apply filtering function to check if a pairs appear in the movies
  support_table_pairs_summed = support_table_pairs.reduceByKey(sum_actors) # sum of values by actor as key
  support_table_pairs_filtered = support_table_pairs_summed.filter(lambda item: item[1] >= threshold) #consider just the actor that performed in more than support value

  return (support_table_pairs_filtered)

#SON algorithm:

It is an exact algorithm that fit well in a context in which data are split in chunks (in our case RDD). 
It consists in applying A-Priori algorithm in each chunks by considering an adjusted threshold, merge the results and then perform a scan on the baskets to identify possible *false positive.*

In our case we are interested in identify the pairs of actors that appear together more than 140 times (minimum threshold = 140)

In [ ]:
#create a list containing baskets: each basket is a movie and it is a list containing the actors that performed in that film
baskets = title_principals.groupBy("tconst").agg(collect_set("nconst").alias("actors"))

#list of baskets containing actors divided by movies
basket_list = baskets.select('actors').rdd.flatMap(list)

#Parallelize the list in RDD
basket_list = sc.parallelize(basket_list.collect(), 5)

print(basket_list.collect()[:2]) #print example

[['nm0029806', 'nm0509573'], ['nm0910564', 'nm0527801', 'nm0399988', 'nm0101071', 'nm0694718', 'nm0728289', 'nm0585503']]


In [ ]:
basket_list.getNumPartitions() #check number of partitions

5

Setting the support and adjust it:

In [ ]:
minSupport = 140
numPartitions = basket_list.getNumPartitions()
adjSupport = minSupport/numPartitions
adjSupport

28.0

Implementation of the SON algorithm procedure by including a-priori algorithm:

In [ ]:
candidates = sc.parallelize([]) #create a RDD to merge the result of apriori applied on each chunk.

for i in range(0, numPartitions-1):
  
  partition = sc.parallelize(basket_list.glom().collect()[i]) #collect each partition
  support_table_pairs_filtered = apriori(partition, adjSupport)
  candidate_chunk = support_table_pairs_filtered.map(lambda item: (item[0],1))
  candidates = candidates.union(candidate_chunk)

In [ ]:
print(candidates.collect()[:5]) #print example

[(('nm0623427', 'nm0006982'), 1), (('nm2082516', 'nm0648803'), 1)]


In [ ]:
#Convert the RDD to dataframe
deptColumns = ["actor_pair","movies"]
candidates_set = candidates.toDF(deptColumns)
candidates_set.createOrReplaceTempView("candidates_set")
sqlContext.sql("SELECT * FROM candidates_set ORDER BY movies DESC").show(truncate = False)

+----------------------+------+
|actor_pair            |movies|
+----------------------+------+
|{nm0623427, nm0006982}|1     |
|{nm2082516, nm0648803}|1     |
+----------------------+------+



Obtain the codes of the candidate actors:

In [ ]:
candidates_actors = candidates.map(lambda item : item[0])
candidates_list = candidates_actors.collect()

Check for possible false positive by scanning the baskets:

In [ ]:
candidates_check = basket_list.map(lambda x : filtering(x, candidates_list)).filter(lambda x: x is not None).cache()
candidates_summed = candidates_check.reduceByKey(sum_actors)
candidates_filtered = candidates_summed.filter(lambda item: item[1] >= minSupport)

print(candidates_filtered.collect())

[(('nm2082516', 'nm0648803'), 147), (('nm0623427', 'nm0006982'), 236)]


Print the results:

In [ ]:
#convert the  RDD to dataframe
deptColumns = ["actor_pair","movies"]
results = candidates_filtered.toDF(deptColumns)
results.createOrReplaceTempView("results")
sqlContext.sql("SELECT * FROM results ORDER BY movies DESC").show(truncate = False)

+----------------------+------+
|actor_pair            |movies|
+----------------------+------+
|{nm0623427, nm0006982}|236   |
|{nm2082516, nm0648803}|147   |
+----------------------+------+



#Results:

To calculate the confidence we need to obtain the number of movies each of these actors performed singularly. 

In [ ]:
#list of all actors    
flat_list  = basket_list.flatMap(list)

#obtain the actors that performed in a number of movies larger than the threshold:
singleton = flat_list.map(lambda item: (item , 1)) #add one for each actor appearence
singleton_summed = singleton.reduceByKey(sum_actors) #sum of values by actor as key
singleton_filtered = singleton_summed.filter(lambda item: item[1] >= minSupport ) #consider just the actor that performed in more than support value

#convert the first support RDD to dataframe
deptColumns = ["actors", "movies"]
first_df = singleton_filtered.toDF(deptColumns)
first_df.createOrReplaceTempView("first_df")
sqlContext.sql("SELECT * FROM first_df ORDER BY movies DESC").show(5, truncate = False)

+---------+------+
|actors   |movies|
+---------+------+
|nm0103977|797   |
|nm0006982|585   |
|nm0648803|565   |
|nm0305182|506   |
|nm0623427|436   |
+---------+------+
only showing top 5 rows



We combine this information with the name_basics table to obtain the name of the actors:

In [ ]:
name_basics.createOrReplaceTempView("name_basics")

#retrieve the name of the most frequent actors from the name_basics table
sqlContext.sql("SELECT primaryName, actors, movies  FROM (first_df INNER JOIN name_basics ON actors = nconst) WHERE actors = 'nm2082516'").show(truncate = False)
sqlContext.sql("SELECT primaryName, actors, movies  FROM (first_df INNER JOIN name_basics ON actors = nconst) WHERE actors = 'nm0648803'").show(truncate = False)

sqlContext.sql("SELECT primaryName, actors, movies  FROM (first_df INNER JOIN name_basics ON actors = nconst) WHERE actors = 'nm0623427'").show(truncate = False)
sqlContext.sql("SELECT primaryName, actors, movies  FROM (first_df INNER JOIN name_basics ON actors = nconst) WHERE actors = 'nm0006982'").show(truncate = False)

+------------+---------+------+
|primaryName |actors   |movies|
+------------+---------+------+
|Kijaku Ôtani|nm2082516|161   |
+------------+---------+------+

+----------------+---------+------+
|primaryName     |actors   |movies|
+----------------+---------+------+
|Matsunosuke Onoe|nm0648803|565   |
+----------------+---------+------+

+-----------+---------+------+
|primaryName|actors   |movies|
+-----------+---------+------+
|Prem Nazir |nm0623427|436   |
+-----------+---------+------+

+-----------+---------+------+
|primaryName|actors   |movies|
+-----------+---------+------+
|Adoor Bhasi|nm0006982|585   |
+-----------+---------+------+



It is possible to calculate the confidence that says how likely actor Y appears when actor X also appears: Confidence{X -> Y}.

In [ ]:
#Confidence computation:
#Confidence that given Otani there is also Onoe = Confidence{Otani -> Onoe} and viceversa
confidence_OT_ON = round(147/161, 2)
confidence_ON_OT = round(147/565, 2)

#Confidence that given Bhasi there is also Nazir and viceversa
confidence_BH_NA = round(236/585, 2)
confidence_NA_BH = round(236/436, 2)

print("The probabilty that Onoe appears in a movie given the presence of Otani is {}.\nThe probability that Otani appears in a movie given the presence of Onoe is {}".format(confidence_OT_ON, confidence_ON_OT))
print("The probabilty that Nazir appears in a movie given the presence of Bhasi is {}.\nThe probability that Bhasi appears in a movie given the presence of Nazir is {}".format(confidence_BH_NA, confidence_NA_BH))

The probabilty that Onoe appears in a movie given the presence of Otani is 0.91.
The probability that Otani appears in a movie given the presence of Onoe is 0.26
The probabilty that Nazir appears in a movie given the presence of Bhasi is 0.4.
The probability that Bhasi appears in a movie given the presence of Nazir is 0.54
